The code is based on https://medium.com/analytics-vidhya/gpt2-for-sentiment-analysis-38cd9832d5e9

Modified to use the full IMDB of movie reviews instead of the 50 examples file.

# **Import Libraries**
Tensorflow 1.x is used because of dependencies needed with gpt_2_simple.

In [ ]:
%tensorflow_version 1.x
!pip install gpt-2-simple 
import json
import pandas as pd
import os
import numpy as np
import tensorflow as tf
import gpt_2_simple as gpt2

# **Import Dataset**

In [ ]:
URL = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file(fname="aclImdb_v1.tar.gz", 
                                  origin=URL,
                                  untar=True,
                                  cache_dir='.',
                                  cache_subdir='')

# **Preprocessing**
The dataset chosed has tre principal folders: one contains files of positive reviews, one that contains files of negative reviews and a file of unlabeled reviews.
We chose to exclude the unlabeled reviews and divide the labeled reviews in train and test sets, with no validation set. The objective is to hope that gpt2 learns to complete a sentence formatted as:

// *review* || This review is *label* 

where *label* should be completed by gpt. 

The initial preprocessing of the files is inspired by https://towardsdatascience.com/sentiment-analysis-in-10-minutes-with-bert-and-hugging-face-294e8a04b671




In [ ]:
# The shutil module offers a number of high-level 
# operations on files and collections of files.
import os
import shutil
# Create main directory path ("/aclImdb")
main_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
# Create sub directory path ("/aclImdb/train")
train_dir = os.path.join(main_dir, 'train')
# Remove unsup folder since this is a supervised learning task
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)
# View the final train folder
print(os.listdir(train_dir))

In [ ]:
dir_file_pos = os.listdir(train_dir+"/pos")
print("Positivi: ") 
print(dir_file_pos)
dir_file_neg = os.listdir(train_dir+"/neg")
print("Negativi: ")
print(dir_file_neg)

In [5]:
dataset = []
pos_dir = os.path.join(train_dir,'pos')
neg_dir = os.path.join(train_dir,'neg')

for element in dir_file_pos:
  input_file = open(pos_dir + "/"+ element, "r")
  line = input_file.readline()
  dataset.append("// " + line + " || This review is " + "Positive" + "\n")
  input_file.close()

for element in dir_file_neg:
  input_file = open(neg_dir + "/"+ element, "r")
  line = input_file.readline()
  dataset.append("// " + line + " || This review is " + "Negative" + "\n")
  input_file.close()

#dataset

In [6]:
import random
random.shuffle(dataset)
#dataset

In [ ]:
len(dataset)

In [8]:
count = 0
train = []
while count < 20000:
  train.append(dataset[count])
  count = count + 1

count = 20000
test = []
while count < 25000:
    test.append(dataset[count])
    count = count + 1
#train

In [9]:
#test

In [9]:
file_train = open("train.txt", "w")
for element in train:
  file_train.write(element)
file_train.close()

In [11]:
#useless
#file_test = open("test.txt", "w")
#for element in test:
#  file_test.write(element)
#file_test.close()

# **Downloading and Training the model**

In [ ]:
#model_name = "117M"
model_name = "345M" # The GPT-2 model we're using
#model_name = "774M" #OOM, it was fun until it lasted
gpt2.download_gpt2(model_name=model_name) # Download the model

In [ ]:
train_path = "train.txt"
sess = gpt2.start_tf_sess()
gpt2.finetune(sess, #session
              train_path, #path string of the file where the training set is saved
              model_name=model_name, #used model 
              run_name='run1', # no idea i hope this works
              checkpoint_dir='checkpoint', #same
              steps=2)   # max number of training steps


# **Interaction with the model and results**
Code reference that inspired this section:

https://github.com/minimaxir/gpt-2-simple/blob/master/gpt_2_simple/gpt_2.py

https://github.com/openai/gpt-2/blob/master/src/generate_unconditional_samples.py

https://github.com/openai/gpt-2/blob/master/src/interactive_conditional_samples.py

https://colab.research.google.com/drive/1ulO-Z0G6BdvQAZ83PJNXCS0ygtRtp46g




In [12]:
#no idea what this does??? but has to be executed 
!git clone https://github.com/openai/gpt-2.git
import os
os.chdir("gpt-2/src/")
import tensorflow as tf
import model, sample, encoder
os.chdir('../../')

Cloning into 'gpt-2'...
remote: Enumerating objects: 233, done.
remote: Total 233 (delta 0), reused 0 (delta 0), pack-reused 233
Receiving objects: 100% (233/233), 4.38 MiB | 21.25 MiB/s, done.
Resolving deltas: 100% (124/124), done.


In [13]:
samples = open("samples.txt","w")
samples.write("Positive"+"\n")
samples.write("Negative"+"\n")
samples.close()

In [ ]:
correct = 0
n_tests = 5000
for element in test:
  print(element + "\n")
  split = element.split("|| This review is ")
  input = split[0].strip()
  #print(input + "\n")
  input = input + " ||  This review is "
  #print(input + "\n")
  expected_output = split[1].strip()
  gpt2.generate(sess,
             run_name='run1',
             checkpoint_dir='checkpoint',
             model_name=model_name,
             model_dir='models',
             sample_dir='samples',
             return_as_list=False,
             truncate=None,
             destination_path=None,
             sample_delim='\n',
             prefix=input,
             seed=None,
             nsamples=1,
             batch_size=1,
             length=8,
             temperature=0.7,
             top_k=0,
             top_p=0.0,
             include_prefix=True)

  print("Expected: " + expected_output + ", Answer: ") #+ str(out) + "\n")
  #if (str(out).casefold() == expected_output.casefold()):
    #correct = correct + 1
print("Correct " + str(correct) + " out of" + str(n_tests) + "\n")


In [14]:
def interact_model(
    model_name,
    seed,
    nsamples,
    batch_size,
    length,
    temperature,
    top_k,
    models_dir
):
    models_dir = os.path.expanduser(os.path.expandvars(models_dir))
    if batch_size is None:
        batch_size = 1
    assert nsamples % batch_size == 0

    enc = encoder.get_encoder(model_name, models_dir)
    hparams = model.default_hparams()
    with open(os.path.join(models_dir, model_name, 'hparams.json')) as f:
        hparams.override_from_dict(json.load(f))

    if length is None:
        length = hparams.n_ctx // 2
    elif length > hparams.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

    with tf.Session(graph=tf.Graph()) as sess:
        context = tf.placeholder(tf.int32, [batch_size, None])
        np.random.seed(seed)
        tf.set_random_seed(seed)
        output = sample.sample_sequence(
            hparams=hparams, length=length,
            context=context,
            batch_size=batch_size,
            temperature=temperature, top_k=top_k
        )

        saver = tf.train.Saver()
        ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))
        saver.restore(sess, ckpt)

#        while True:
#            raw_text = input("Model prompt >>> ")
#            while not raw_text:
#                print('Prompt should not be empty!')
#                raw_text = input("Model prompt >>> ")
#            context_tokens = enc.encode(raw_text)
#            generated = 0
#            for _ in range(nsamples // batch_size):
#                out = sess.run(output, feed_dict={
#                    context: [context_tokens for _ in range(batch_size)]
#                })[:, len(context_tokens):]
#                for i in range(batch_size):
#                    generated += 1
#                    text = enc.decode(out[i])
#                    print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
#                    print(text)
#            print("=" * 80)
        n_tests = 5000
        correct = 0
        for element in test:
          split = element.split("||  This review is")
          input = split[0].strip()
          #print(input + "\n")
          input = input + " ||  This review is "
          print(input + "\n")
          expected_output = split[1].strip()
          context_tokens = enc.encode(input)
          #due to a bug https://github.com/minimaxir/gpt-2-simple/issues/38
          #long long reviews are excluded
          try: 
            for _ in range(nsamples // batch_size):
                out = sess.run(output, feed_dict={
                    context: [context_tokens for _ in range(batch_size)]
                })[:, len(context_tokens):]
                for i in range(batch_size):
                    text = enc.decode(out[i])
                    print("Expected: " + expected_output + ", Answer: " + text + "\n")
                    if (text.casefold() == expected_output.casefold()):
                      correct = correct + 1
          except:
            print("-------------- too long -------------")
            n_tests = n_tests - 1
        print("Correct " + str(correct) + " out of" + str(n_tests) + "\n")

In [ ]:
#Test
!ls
interact_model(
    'run1',
    None,
    1,
    1,
    2,
    1,
    0,
    './checkpoint'
)